In [1]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('C:/Users/luizf/Desktop/MINDS/Datasets/air_quality_beijing.csv', sep=',')
data = data.drop(labels=['No','day','year','month','hour','wd','station'], axis=1)
data = data.drop(labels=['No.1','day.1','year.1','month.1','hour.1','wd.1','station.1'], axis=1)
data = data.drop(labels=['No.2','day.2','year.2','month.2','hour.2','wd.2','station.2'], axis=1)
data = data.drop(labels=['No.3','day.3','year.3','month.3','hour.3','wd.3','station.3'], axis=1)
data = data.drop(labels=['No.4','day.4','year.4','month.4','hour.4','wd.4','station.4'], axis=1)
data = data.drop(labels=['No.5','day.5','year.5','month.5','hour.5','wd.5','station.5'], axis=1)
data = data.drop(labels=['No.6','day.6','year.6','month.6','hour.6','wd.6','station.6'], axis=1)
data = data.drop(labels=['No.7','day.7','year.7','month.7','hour.7','wd.7','station.7'], axis=1)
data = data.drop(labels=['No.8','day.8','year.8','month.8','hour.8','wd.8','station.8'], axis=1)
data = data.drop(labels=['No.9','day.9','year.9','month.9','hour.9','wd.9','station.9'], axis=1)
data = data.drop(labels=['No.10','day.10','year.10','month.10','hour.10','wd.10','station.10'], axis=1)
data = data.drop(labels=['No.11','day.11','year.11','month.11','hour.11','wd.11','station.11'], axis=1)
data.dropna(inplace=True)
#data.loc[:,['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH']] = data.loc[:,['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH']].applymap(lambda x: str(x.replace(',','.')))
data = data.astype(float)
X=data.copy()
X.drop(data.tail(1).index,inplace=True)
X
y=data.copy()
y.loc[max(y.index)+1, :] = None
y=y.shift(-1)
y.dropna(inplace=True)
y

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.11,SO2.11,NO2.11,CO.11,O3.11,TEMP.11,PRES.11,DEWP.11,RAIN.11,WSPM.11
23,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,10.0,11.0,35.0,400.0,39.0,0.5,1030.2,-18.3,0.0,1.4
24,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,8.0,9.0,33.0,400.0,40.0,-0.3,1030.2,-17.5,0.0,1.0
25,3.0,7.0,21.0,49.0,500.0,43.0,-0.4,1029.6,-17.6,0.0,...,7.0,17.0,53.0,500.0,17.0,0.0,1028.8,-16.8,0.0,1.3
32,10.0,14.0,47.0,62.0,700.0,29.0,0.6,1029.7,-16.7,0.0,...,11.0,17.0,50.0,500.0,23.0,1.1,1028.9,-16.8,0.0,1.0
33,11.0,19.0,36.0,60.0,600.0,34.0,1.7,1028.9,-16.3,0.0,...,14.0,22.0,52.0,600.0,20.0,1.9,1028.4,-17.1,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35051,6.0,13.0,3.0,10.0,200.0,112.0,12.9,1015.4,-14.9,0.0,...,15.0,2.0,9.0,300.0,77.0,12.9,1015.4,-14.9,0.0,3.3
35052,6.0,8.0,3.0,9.0,100.0,115.0,14.0,1014.5,-15.0,0.0,...,8.0,2.0,7.0,300.0,82.0,14.0,1014.5,-15.0,0.0,2.9
35053,16.0,37.0,10.0,66.0,700.0,58.0,10.8,1014.2,-13.3,0.0,...,28.0,4.0,38.0,500.0,54.0,10.8,1014.2,-13.3,0.0,1.1
35061,21.0,44.0,12.0,87.0,700.0,35.0,10.5,1014.4,-12.9,0.0,...,23.0,4.0,30.0,400.0,59.0,10.5,1014.4,-12.9,0.0,1.2


In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train, X_test = train_test_split(X, test_size=0.25, shuffle=False)
X_train, X_val = train_test_split(X_train, test_size=0.15, shuffle=False)
X_train = scaler.fit_transform(X_train.values)
X_val = scaler.fit_transform(X_val.values)
train_loader = torch.utils.data.DataLoader(torch.tensor(X_train.astype(np.float32)), batch_size=32)
val_loader = torch.utils.data.DataLoader(torch.tensor(X_val.astype(np.float32)), batch_size=32)

y_train, y_test = train_test_split(y, test_size=0.25, shuffle=False)
y_train, y_val = train_test_split(y_train, test_size=0.15, shuffle=False)
y_train = scaler.fit_transform(y_train.values)
y_val = scaler.fit_transform(y_val.values)

In [4]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(GRUModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # GRU layers
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        if torch.cuda.is_available():
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().cuda()
        else:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)

        return out

In [5]:
def train_gru(X_train, layer_dim, dropout_prob, epochs, batch_size, learning_rate, step_wdecay, gamma_wdecay, X_val=0):
    train_loader = torch.utils.data.DataLoader(torch.tensor(X_train.astype(np.float32)), batch_size=batch_size, shuffle=False)
    #val_loader = torch.utils.data.DataLoader(torch.tensor(X_val.astype(np.float32)), batch_size=batch_size,shuffle=False)
    if torch.cuda.is_available():
        y_train_t= torch.tensor(y_train.astype(np.float32)).cuda()
        #y_val_t = torch.tensor(y_val.astype(np.float32)).cuda()
    else:
        y_train_t = torch.tensor(y_train.astype(np.float32))
        #y_val_t = torch.tensor(y_val.astype(np.float32))
    torch.manual_seed(1)
    alpha = 0.5
    h=round(X_train.shape[0]/(alpha * (X_train.shape[1]*2)))
    model = GRUModel(X_train.shape[1],h, layer_dim,X_train.shape[1],dropout_prob)
    if torch.cuda.is_available():
        model = model.cuda()
    else:
        model = model.cuda()
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    buffer = 10000
    steplr = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_wdecay, gamma=gamma_wdecay)

    min_valid_loss=500
    for epoch in range(epochs):
        aux,aux2=0,0
        train_loss=0.0
        model.train()
        for batch, d in enumerate(train_loader):
            if torch.cuda.is_available():
                d =d.cuda()
            optimizer.zero_grad()  #set gradient to 0
            output = model(d.reshape(-1, d.shape[1]).unsqueeze(1))  #make a prediction
            if torch.cuda.is_available():
                loss = criterion(output, y_train_t[aux:aux+d.shape[0]])
            else:
                loss = criterion(output, torch.tensor(y_train_t[aux:aux+d.shape[0]].astype(np.float32)))  #calculate the loss
            loss.backward()  #differentiate loss with respect to parameters
            optimizer.step() #apply the method step and updates the parameters 
            train_loss += loss.item()
            aux+=d.shape[0]
        steplr.step()

        #valid_loss = 0.0
        #model.eval() 
        #for dv in val_loader:  
        #    if torch.cuda.is_available():
        #        dv =dv.cuda()  
        #    target = model(dv.unsqueeze(1))
        #    if torch.cuda.is_available():
        #        val_loss = criterion(target, y_val_t[aux2:aux2+dv.shape[0]])
        #    else:
        #        val_loss = criterion(target,torch.tensor(y_val_t[aux2:aux2+dv.shape[0]].astype(np.float32)))
        #    valid_loss += val_loss.item()
        #    aux2+=dv.shape[0]
        #print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(val_loader)}')
        #if valid_loss < buffer:
        #    buffer = valid_loss
#
   # print (buffer/len(val_loader))   
    return model

In [6]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [7]:
n_windows = 30
w = round(len(X)/n_windows)
buffer =  {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape":[],
         "nrmse":[],
         "variable":[]
    }
    
col = ["window","rmse","mape","mae","r2","variable"]
result = pd.DataFrame(columns=col)
a={}
for i in range(n_windows):
    dfx=X.iloc[int(w*i):int(w*(i+1)), :]
    dfy=y.iloc[int(w*i):int(w*(i+1)), :]
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_train, X_test = train_test_split(dfx, test_size=0.25, shuffle=False)
    #X_train, X_val = train_test_split(X_train, test_size=0.15, shuffle=False)
    X_train = scaler.fit_transform(X_train.values)
    #X_val = scaler.fit_transform(X_val.values)


    y_train, y_test = train_test_split(dfy, test_size=0.25, shuffle=False)
    #y_train, y_val = train_test_split(y_train, test_size=0.15, shuffle=False)
    y_train = scaler.fit_transform(y_train.values)
    #y_val = scaler.fit_transform(y_val.values)

    model = train_gru(X_train, layer_dim=1, dropout_prob=0, epochs=300, batch_size=64, learning_rate=0.001, step_wdecay=50, gamma_wdecay=0.5)

    model.eval()
    if torch.cuda.is_available():
        test_norm= torch.tensor(scaler.fit_transform(X_test.values.astype(np.float32))).cuda()
    else:
        test_norm= torch.tensor(scaler.fit_transform(X_test.values.astype(np.float32)))
    y_norm = scaler.fit_transform(y_test.values)
    #yhat=scaler.inverse_transform(model(test_norm.unsqueeze(1)).detach().numpy())
    yhat=scaler.inverse_transform(model(test_norm.unsqueeze(1)).detach().cpu().numpy())
    #y=scaler.inverse_transform(test_norm.detach().numpy())[:,0]
    
    from pyFTS.benchmarks import Measures
    buffer['rmse']=[Measures.rmse(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['mape']=[Measures.mape(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['mae']=[mean_absolute_error(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['r2']=[r2_score(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['smape']=[Measures.smape(y_test.values[:,j],yhat[:,j]) for j in range (X.shape[1])]
    buffer['nrmse']=[cal_nrmse(Measures.rmse(y_test.values[:,j],yhat[:,j]),y_test.values[:,j]) for j in range (X.shape[1])]
    buffer['variable']=[j for j in X.columns]
    buffer['window']= w*i

    result = pd.concat([result, pd.DataFrame(buffer)], ignore_index=True)
    #result['rmse'].append([j for j in buffer['rmse']])
    #result['mape'].append(buffer['mape'])
    #result['mae'].append(buffer['mae'])
    #result['r2'].append(buffer['r2'])
    #result['variable'].append([j for j in X.columns])
    #result['window'].append(buffer['window'])
#
#result = np.mean(buffer)
#print (result)


    
    


C:\Users\luizf\miniconda3\envs\test\lib\site-packages\pyFTS\benchmarks\Measures.py:71: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
C:\Users\luizf\AppData\Local\Temp/ipykernel_27464/1568283450.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  return (rmse/x)
C:\Users\luizf\miniconda3\envs\test\lib\site-packages\pyFTS\benchmarks\Measures.py:71: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
C:\Users\luizf\miniconda3\envs\test\lib\site-packages\pyFTS\benchmarks\Measures.py:71: RuntimeWarning: divide by zero encountered in true_divide
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
C:\Users\luizf\AppData\Local\Temp/ipykernel_27464/1568283450.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  return (rmse/x)
C:\Users\lui

In [8]:
result

,window,rmse,mape,mae,r2,variable,smape,nrmse
0,0,39.751474,51.775053,28.334570,0.542256,PM2.5,20.694946,0.154075
1,0,53.038714,46.314668,38.745753,0.402012,PM10,18.596012,0.180404
2,0,14.814690,70.059461,9.907703,0.197546,SO2,24.125521,0.148147
3,0,26.777169,52.811273,20.258115,-0.150850,NO2,19.477492,0.225018
4,0,267.403708,27.875761,212.681136,0.415324,CO,13.049220,0.178269
...,...,...,...,...,...,...,...,...
3955,16559,4.787402,421.293039,4.157036,0.058709,TEMP.11,56.185396,0.270475
3956,16559,4.830568,0.342171,3.482733,0.376128,PRES.11,0.171039,0.171297
3957,16559,4.229083,35.139355,3.543623,0.376002,DEWP.11,14.782846,0.221418
3958,16559,0.095537,inf,0.033420,-0.054122,RAIN.11,99.900161,0.119421


In [9]:
final_measures = {'variable':[], 'rmse': [], 'mae': [], 'mape': [], 'r2': [], 'smape':[], 'nrmse':[], 'rmse_std': [], 'mae_std': [], 'mape_std': [], 'r2_std': [], 'smape_std':[], 'nrmse_std':[]}
m=[]
for i in list(X.columns):
    m=result[result.variable==i].mean()
    st = result[result.variable==i].std()
    final_measures['variable'].append(i)
    final_measures['rmse'].append(m['rmse'])
    final_measures['mae'].append(m['mae'])
    final_measures['mape'].append(m['mape'])
    final_measures['r2'].append(m['r2'])
    final_measures['smape'].append(m['smape'])
    final_measures['nrmse'].append(m['nrmse'])
    final_measures['rmse_std'].append(st['rmse'])
    final_measures['mae_std'].append(st['mae'])
    final_measures['mape_std'].append(st['mape'])
    final_measures['r2_std'].append(st['r2'])
    final_measures['smape_std'].append(st['smape'])
    final_measures['nrmse_std'].append(st['nrmse'])
pd.DataFrame(final_measures)

C:\Users\luizf\AppData\Local\Temp/ipykernel_27464/2831257117.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  m=result[result.variable==i].mean()
C:\Users\luizf\AppData\Local\Temp/ipykernel_27464/2831257117.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  st = result[result.variable==i].std()


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5,40.823331,31.616379,111.129769,0.499845,27.450848,0.163957,21.059333,16.821416,82.965969,0.198814,8.556570,0.024452
1,PM10,51.089481,39.308092,88.139923,0.417879,23.810723,0.165548,22.292498,16.157972,45.276096,0.208166,6.090483,0.021176
2,SO2,10.679604,7.770666,82.702885,0.155331,26.359348,0.192538,9.463944,7.317045,41.373071,0.171127,6.534713,0.038574
3,NO2,24.245226,19.234614,57.974190,0.277542,18.964287,0.192219,8.593401,7.223912,25.340734,0.284578,4.621435,0.031901
4,CO,578.251526,420.293770,50.002442,0.437867,19.061521,0.162541,365.957535,257.994790,26.560326,0.208738,6.943493,0.024304
...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,TEMP.11,191.968635,124.269191,inf,0.168200,31.714611,0.205709,729.418832,461.810406,NaN,0.203784,30.025897,0.039373
128,PRES.11,2980.805758,1295.257983,105.403341,0.248820,3.212534,0.202012,16305.398820,7077.640290,575.664601,0.187154,16.769499,0.039573
129,DEWP.11,3.683912,2.949650,inf,0.441892,20.254772,0.178840,1.732666,1.363880,NaN,0.210055,15.713527,0.033690
130,RAIN.11,0.334838,0.160185,inf,-0.007027,98.838439,inf,0.627712,0.256775,NaN,0.020251,2.104343,NaN


In [10]:
pd.DataFrame(final_measures).to_csv('gru_beijing12_result.csv')